In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gzip
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [2]:
def readJSON(path):
    for l in gzip.open(path, 'r+'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u,g,d
    
def parseData(path):
    for l in gzip.open(path, 'r+'):
        yield eval(l)

In [14]:
listings_summary = pd.read_csv('listings.csv')
listings_detail = pd.read_csv('listings_detail.csv')
reviews_summary = pd.read_csv('reviews.csv')
reviews_detail = pd.read_csv('reviews_detail.csv')
calendar = pd.read_csv('calendar.csv')
neighborhoods = pd.read_csv('neighbourhoods.csv')

In [15]:
listings_summary

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,5570,Ocean front condo on the sand,8435,Jef Karchin,NaN,Mission Bay,32.784300,-117.252580,Entire home/apt,2050,3,0,NaN,NaN,3,197
1,29967,"Great home, 10 min walk to Beach",129123,Michael,NaN,Pacific Beach,32.807240,-117.256300,Entire home/apt,225,4,61,2020-08-31,0.49,5,334
2,38245,Point Loma: Den downstairs,164137,Melinda,NaN,Roseville,32.742020,-117.218700,Private room,74,1,143,2019-10-20,1.16,3,320
3,54001,"La Jolla 2 Bdr Cottage: 1 Qn; 2Twns, Blks 2 Ocn",252692,Marsha,NaN,La Jolla,32.813010,-117.268560,Entire home/apt,135,3,234,2020-10-18,1.96,2,102
4,62274,"charming, colorful, close to beach",302986,Isabel,NaN,Pacific Beach,32.807340,-117.242430,Entire home/apt,69,1,609,2020-09-27,5.06,2,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8683,46028900,Amazing apt in Pacific Beach (Crown Point area),69059229,Mariana,NaN,Pacific Beach,32.797710,-117.229690,Entire home/apt,70,4,0,NaN,NaN,1,9
8684,46035696,Gaslamp District Luxury High Rise,102202334,Dequan,NaN,East Village,32.713790,-117.150310,Entire home/apt,214,1,0,NaN,NaN,2,162
8685,46036151,Gas lamp San Diego High-Rise,102202334,Dequan,NaN,East Village,32.711490,-117.151210,Entire home/apt,214,1,0,NaN,NaN,2,162
8686,46037988,South Park Modern Retreat!,2232295,Sara,NaN,Balboa Park,32.723900,-117.131330,Entire home/apt,286,2,0,NaN,NaN,1,364


In [16]:
listings_detail

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,https://www.airbnb.com/rooms/5570,20201025051130,2020-10-25,Ocean front condo on the sand,Missionbeachretreat. Great Ocean Front Condo...,While some may compare San Diego’s Mission Bea...,https://a0.muscache.com/pictures/d3342744-140a...,8435,https://www.airbnb.com/users/show/8435,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,https://www.airbnb.com/rooms/29967,20201025051130,2020-10-25,"Great home, 10 min walk to Beach","This 2 bedroom, 2 full bathroom home offers al...",NaN,https://a0.muscache.com/pictures/76103867/c9b5...,129123,https://www.airbnb.com/users/show/129123,...,10.0,10.0,10.0,NaN,f,5,5,0,0,0.49
2,38245,https://www.airbnb.com/rooms/38245,20201025051130,2020-10-25,Point Loma: Den downstairs,Stay in a 1928 Spanish Style home. Den is dow...,"This neighborhood is an expensive, high-end Po...",https://a0.muscache.com/pictures/f3073822-e493...,164137,https://www.airbnb.com/users/show/164137,...,10.0,9.0,9.0,NaN,f,3,0,3,0,1.16
3,54001,https://www.airbnb.com/rooms/54001,20201025051130,2020-10-25,"La Jolla 2 Bdr Cottage: 1 Qn; 2Twns, Blks 2 Ocn","Quiet, off the beaten path, this secluded cot...","Tranquility, stately trees, meditative, zen-fu...",https://a0.muscache.com/pictures/47035b04-633c...,252692,https://www.airbnb.com/users/show/252692,...,10.0,10.0,10.0,NaN,f,2,1,1,0,1.96
4,62274,https://www.airbnb.com/rooms/62274,20201025051130,2020-10-25,"charming, colorful, close to beach","Modern, cozy, artsy, and european in a lush ga...","We are in walking distance to Starbucks, Trade...",https://a0.muscache.com/pictures/84577734/93ca...,302986,https://www.airbnb.com/users/show/302986,...,10.0,10.0,10.0,NaN,f,2,2,0,0,5.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8683,46028900,https://www.airbnb.com/rooms/46028900,20201025051130,2020-10-25,Amazing apt in Pacific Beach (Crown Point area),Hello there . My name is Mariana . I have an 2...,NaN,https://a0.muscache.com/pictures/c7db29d8-f13b...,69059229,https://www.airbnb.com/users/show/69059229,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
8684,46035696,https://www.airbnb.com/rooms/46035696,20201025051130,2020-10-25,Gaslamp District Luxury High Rise,NaN,NaN,https://a0.muscache.com/pictures/2b684b36-1d3e...,102202334,https://www.airbnb.com/users/show/102202334,...,NaN,NaN,NaN,NaN,t,2,2,0,0,NaN
8685,46036151,https://www.airbnb.com/rooms/46036151,20201025051130,2020-10-25,Gas lamp San Diego High-Rise,NaN,NaN,https://a0.muscache.com/pictures/b2232b80-92f5...,102202334,https://www.airbnb.com/users/show/102202334,...,NaN,NaN,NaN,NaN,t,2,2,0,0,NaN
8686,46037988,https://www.airbnb.com/rooms/46037988,20201025051130,2020-10-25,South Park Modern Retreat!,"Welcome to South Park, one of San Diego's olde...",This is the best neighborhood in San Diego! As...,https://a0.muscache.com/pictures/79e51e22-d1d8...,2232295,https://www.airbnb.com/users/show/2232295,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN


In [9]:
reviews_summary

,listing_id,date
0,29967,2010-07-09
1,29967,2010-07-14
2,29967,2010-07-22
3,29967,2010-07-29
4,29967,2010-08-07
...,...,...
418381,45879403,2020-10-16
418382,45879403,2020-10-17
418383,45879403,2020-10-18
418384,45879403,2020-10-19


In [10]:
reviews_detail

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,29967,62788,2010-07-09,151260,Debbie,When I booked our stay in San Diego at Dennis ...
1,29967,64568,2010-07-14,141552,Eric,This was my first experience with using airbnb...
2,29967,67502,2010-07-22,141591,David,We found the house to be very accommodating--e...
3,29967,70466,2010-07-29,125982,Anders,As advertised and more. Dennis was very helpfu...
4,29967,74876,2010-08-07,29835,Miyoko,We had a great time in San Diego. Denis' house...
...,...,...,...,...,...,...
418381,45879403,700933618,2020-10-16,356204795,Shanna,Jeffrey and his girlfriend were outstanding ho...
418382,45879403,701234991,2020-10-17,57040410,Liz,This place was perfect for me! It was really s...
418383,45879403,701918134,2020-10-18,310817121,Melissa,This place was beautiful! Didn’t have a doubt....
418384,45879403,702188172,2020-10-19,46954062,Steven,Highly recommend this place for anyone wanting...


In [12]:
calendar

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,5570,2020-10-25,f,"$2,050.00","$2,050.00",3,365
1,5570,2020-10-26,f,"$2,050.00","$2,050.00",3,365
2,5570,2020-10-27,f,"$2,050.00","$2,050.00",3,365
3,5570,2020-10-28,f,"$2,050.00","$2,050.00",3,365
4,5570,2020-10-29,f,"$2,050.00","$2,050.00",3,365
...,...,...,...,...,...,...,...
3170750,45952986,2021-10-20,t,$83.00,$83.00,30,365
3170751,45952986,2021-10-21,t,$83.00,$83.00,30,365
3170752,45952986,2021-10-22,t,$83.00,$83.00,30,365
3170753,45952986,2021-10-23,t,$83.00,$83.00,30,365


In [13]:
neighborhoods

,neighbourhood_group,neighbourhood
0,NaN,Allied Gardens
1,NaN,Alta Vista
2,NaN,Amphitheater And Water Park
3,NaN,Balboa Park
4,NaN,Bario Logan
...,...,...
103,NaN,Village Center
104,NaN,Webster
105,NaN,West University Heights
106,NaN,Wooded Area


In [17]:
pd.merge(listings_summary,listings_detail,how='outer',on='id')

,id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude_x,longitude_x,room_type_x,price_x,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y
0,5570,Ocean front condo on the sand,8435,Jef Karchin,NaN,Mission Bay,32.784300,-117.252580,Entire home/apt,2050,...,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,"Great home, 10 min walk to Beach",129123,Michael,NaN,Pacific Beach,32.807240,-117.256300,Entire home/apt,225,...,10.0,10.0,10.0,NaN,f,5,5,0,0,0.49
2,38245,Point Loma: Den downstairs,164137,Melinda,NaN,Roseville,32.742020,-117.218700,Private room,74,...,10.0,9.0,9.0,NaN,f,3,0,3,0,1.16
3,54001,"La Jolla 2 Bdr Cottage: 1 Qn; 2Twns, Blks 2 Ocn",252692,Marsha,NaN,La Jolla,32.813010,-117.268560,Entire home/apt,135,...,10.0,10.0,10.0,NaN,f,2,1,1,0,1.96
4,62274,"charming, colorful, close to beach",302986,Isabel,NaN,Pacific Beach,32.807340,-117.242430,Entire home/apt,69,...,10.0,10.0,10.0,NaN,f,2,2,0,0,5.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8683,46028900,Amazing apt in Pacific Beach (Crown Point area),69059229,Mariana,NaN,Pacific Beach,32.797710,-117.229690,Entire home/apt,70,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
8684,46035696,Gaslamp District Luxury High Rise,102202334,Dequan,NaN,East Village,32.713790,-117.150310,Entire home/apt,214,...,NaN,NaN,NaN,NaN,t,2,2,0,0,NaN
8685,46036151,Gas lamp San Diego High-Rise,102202334,Dequan,NaN,East Village,32.711490,-117.151210,Entire home/apt,214,...,NaN,NaN,NaN,NaN,t,2,2,0,0,NaN
8686,46037988,South Park Modern Retreat!,2232295,Sara,NaN,Balboa Park,32.723900,-117.131330,Entire home/apt,286,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
